In [1]:
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt

In [2]:
prescriptions = pd.read_csv("05_cleaned_data/prescriptions_wa.csv", low_memory=False)

In [3]:
prescriptions.columns

Index(['REPORTER_DEA_NO', 'REPORTER_BUS_ACT', 'REPORTER_NAME',
       'REPORTER_ADDL_CO_INFO', 'REPORTER_ADDRESS1', 'REPORTER_ADDRESS2',
       'REPORTER_CITY', 'REPORTER_STATE', 'REPORTER_ZIP', 'REPORTER_COUNTY',
       'BUYER_DEA_NO', 'BUYER_BUS_ACT', 'BUYER_NAME', 'BUYER_ADDL_CO_INFO',
       'BUYER_ADDRESS1', 'BUYER_ADDRESS2', 'BUYER_CITY', 'BUYER_STATE',
       'BUYER_ZIP', 'BUYER_COUNTY', 'TRANSACTION_CODE', 'DRUG_CODE', 'NDC_NO',
       'DRUG_NAME', 'QUANTITY', 'UNIT', 'ACTION_INDICATOR', 'ORDER_FORM_NO',
       'CORRECTION_NO', 'STRENGTH', 'TRANSACTION_DATE', 'CALC_BASE_WT_IN_GM',
       'DOSAGE_UNIT', 'TRANSACTION_ID', 'Product_Name', 'Ingredient_Name',
       'Measure', 'MME_Conversion_Factor', 'Combined_Labeler_Name',
       'Revised_Company_Name', 'Reporter_family', 'dos_str', 'Year', 'Month',
       'StateFIPS', 'CountyName', 'StateName', 'CountyFIPS', 'StateAbbr',
       'STATE_COUNTY', 'state_abbrev', 'FIP_unique', 'State', 'Population',
       'county_test'],
      dtyp

In [4]:
prescriptions["QUANTITY"].describe()

count    30679.000000
mean         2.819551
std          8.179319
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max        300.000000
Name: QUANTITY, dtype: float64

In [5]:
prescriptions_reduced = prescriptions[['QUANTITY', 'Year', 'StateFIPS', 'StateName', 'CountyFIPS', 'FIP_unique', 'Population','county_test']]

In [6]:
prescriptions_reduced["StateName"].value_counts()

Oregon        13622
Washington     4727
Montana        4467
Idaho          3477
Nevada         2777
Wyoming        1609
Name: StateName, dtype: int64

In [7]:
washington = prescriptions_reduced[prescriptions_reduced["StateName"] == "Washington"]
comp = prescriptions_reduced[prescriptions_reduced["StateName"] != "Washington"]

In [8]:
wa_prescriptions = washington.copy()
comp_prescriptions = comp.copy()

In [9]:
wa_prescriptions["quantity_sum"] = wa_prescriptions.groupby(["Year", "FIP_unique"])["QUANTITY"].transform('sum')
comp_prescriptions["quantity_sum"] = comp_prescriptions.groupby(["Year", "FIP_unique"])["QUANTITY"].transform('sum')

In [10]:
wa_prescriptions = wa_prescriptions.drop('QUANTITY', axis=1)
comp_prescriptions = comp_prescriptions.drop('QUANTITY', axis=1)

In [11]:
wa_result = wa_prescriptions.drop_duplicates()
comp_result = comp_prescriptions.drop_duplicates()

In [12]:
wa_result["shipment_per_100k"] = wa_result["quantity_sum"] / wa_result["Population"] * 100_000
comp_result["shipment_per_100k"] = comp_result["quantity_sum"] / comp_result["Population"] * 100_000

/var/folders/gr/gh3wfyb105ddr4t6thtzf7nh0000gn/T/ipykernel_55497/3325995965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wa_result["shipment_per_100k"] = wa_result["quantity_sum"] / wa_result["Population"] * 100_000
/var/folders/gr/gh3wfyb105ddr4t6thtzf7nh0000gn/T/ipykernel_55497/3325995965.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_result["shipment_per_100k"] = comp_result["quantity_sum"] / comp_result["Population"] * 100_000


In [13]:
wa_result.groupby("Year")["shipment_per_100k"].agg([np.mean, np.std])

,mean,std
Year,,
2009.0,197.901727,116.083022
2010.0,213.048937,121.636502
2011.0,260.539723,217.501792
2012.0,235.347538,171.433841


In [14]:
wa_result_b4 = wa_result[wa_result["Year"] < 2012]
wa_result_after = wa_result[wa_result["Year"] >= 2012]

In [15]:
# attempt 1
# washington
# https://altair-viz.github.io/user_guide/transform/aggregate.html

source_data = wa_result_b4

plot_wa_b4 = alt.Chart(source_data).mark_point().encode(
    y=alt.Y("mean_shipment:Q", scale=alt.Scale(zero=False)),
    x=alt.X("Year:O", scale=alt.Scale(zero=False))
).transform_aggregate(
    mean_shipment='mean(shipment_per_100k)',
    groupby=["Year"]
)

plot_wa_b4

alt.Chart(...)

In [16]:
fit_wa_b4 = plot_wa_b4.transform_regression('Year', 'mean_shipment',method="linear"
).mark_line()

fit_wa_b4

alt.Chart(...)

In [17]:
source_data = wa_result_after

plot_wa_after = alt.Chart(source_data).mark_point().encode(
    y=alt.Y("mean_shipment:Q", scale=alt.Scale(zero=False)),
    x=alt.X("Year:O", scale=alt.Scale(zero=False))
).transform_aggregate(
    mean_shipment='mean(shipment_per_100k)',
    groupby=["Year"]
)

plot_wa_after

alt.Chart(...)

In [18]:
fit_wa_after = plot_wa_after.transform_regression('Year', 'mean_overdose',method="linear"
).mark_line()

fit_wa_after

alt.Chart(...)

In [19]:
plot_wa_b4 + fit_wa_b4 + plot_wa_after + fit_wa_after

alt.LayerChart(...)

In [20]:
# comparison states
comp_result_b4 = comp_result[comp_result["Year"] < 2012]
comp_result_after = comp_result[comp_result["Year"] >= 2012]

source_data = comp_result_b4

plot_comp_b4 = alt.Chart(source_data).mark_point().encode(
    y=alt.Y("mean_shipment:Q", scale=alt.Scale(zero=False)),
    x=alt.X("Year:O", scale=alt.Scale(zero=False))
).transform_aggregate(
    mean_shipment='mean(shipment_per_100k)',
    groupby=["Year"]
)

plot_comp_b4
fit_comp_b4 = plot_comp_b4.transform_regression('Year', 'mean_shipment',method="linear"
).mark_line()

fit_comp_b4

source_data = comp_result_after

plot_comp_after = alt.Chart(source_data).mark_point().encode(
    y=alt.Y("mean_shipment:Q", scale=alt.Scale(zero=False)),
    x=alt.X("Year:O", scale=alt.Scale(zero=False))
).transform_aggregate(
    mean_shipment='mean(shipment_per_100k)',
    groupby=["Year"]
)

plot_comp_after

fit_comp_after = plot_comp_after.transform_regression('Year', 'mean_shipment',method="linear"
).mark_line()

fit_comp_after
plot_comp_b4 + fit_comp_b4 + plot_comp_after + fit_comp_after

alt.LayerChart(...)